In [6]:
import requests
import pandas as pd 
from datetime import datetime
import json



# Scrape raw data

In [7]:

url = "https://api.climateview.global/boards/Boards/ec2d0cdf-e70e-43fb-85cb-ed6b31ee1e09/published/v3"
r = requests.get(url)


# Store copy of raw data

In [8]:
json_data = r.json()

today = datetime.now().strftime('%Y-%m-%d')

outfile = f"data/raw/{today}.json"
with open(outfile, "w") as f:
    json.dump(json_data, f, indent=2)
    print(f"=> {outfile}")

=> data/raw/2023-03-26.json


# Parse key data to csv

In [9]:
def get_values(d):
    """Parse non-nested values (no dicts or lists)"""
    values = {}
    for k, v in d.items():
        if isinstance(v, list) or isinstance(v, dict):
            continue
        values[k] = v
    return values


nodes = []
for node_id, node_data in json_data["content"]["entityData"]["nodes"].items():
    node = {
        "id": node_id,
        "title": node_data["title"],
    }
    # parse data
    node.update(get_values(node_data["nodeProperties"]))

    if "customTargetModelData" in node_data["nodeProperties"]:
        # parse more data
        node.update(get_values(node_data["nodeProperties"]["customTargetModelData"]))
        
        # get timeserie data from charts
        chart_data = node_data["nodeProperties"]["customTargetModelData"]["chartData"]
        for k, v in chart_data.items():
            node[k] = json.dumps(v)
        
        nodes.append(node)

outfile = f"data/parsed/{today}.csv"
df_nodes = pd.DataFrame(nodes).set_index("id")
df_nodes.to_csv(outfile)
print(f"=> {outfile}")


=> data/parsed/2023-03-26.csv


In [66]:
d = node_data["nodeProperties"]["customTargetModelData"]["chartData"]

def interpolate(ts_dict):
    s = pd.Series(ts_dict)
    s.index = s.index.astype(int)
    y_min = s.index.min()
    y_max = s.index.max()
    s = s.reindex(range(y_min, y_max + 1))
    return s.interpolate().to_dict()


# 'carbonAbatement'
#    s_target.index = s_target.index.astype(int)
#    years = list(range(s_target.index[0], s_target.index[-1] + 1))
#    s_target = s_target.reindex(years).interpolate()


to_list(interpolate(d['carbonAbatement']))


[(2010, 0.0),
 (2011, 0.0),
 (2012, 0.0),
 (2013, 0.0),
 (2014, 0.0),
 (2015, 0.0),
 (2016, 0.0),
 (2017, 0.0),
 (2018, 0.0),
 (2019, 0.0),
 (2020, 0.0),
 (2021, 0.0),
 (2022, 0.0),
 (2023, 0.0),
 (2024, 0.0),
 (2025, 3.0),
 (2026, 3.0),
 (2027, 3.0),
 (2028, 3.0),
 (2029, 3.0),
 (2030, 4.616),
 (2031, 4.6358),
 (2032, 4.6556),
 (2033, 4.6754),
 (2034, 4.6952),
 (2035, 4.715),
 (2036, 4.7348),
 (2037, 4.7546),
 (2038, 4.7744),
 (2039, 4.7942),
 (2040, 4.814),
 (2041, 4.8338),
 (2042, 4.8536),
 (2043, 4.8734),
 (2044, 4.8932),
 (2045, 4.913)]

In [57]:
d["target"]

{'2010': 0, '2024': 0, '2025': 65, '2029': 65, '2030': 100, '2045': 100}

In [59]:
pd.Series({2020: 1, 2021: None, 2023: 8}).reindex([2020,2021, 2022, 2023]).interpolate()

2020    1.000000
2021    3.333333
2022    5.666667
2023    8.000000
dtype: float64

In [5]:
# parse indicator data
indicators = []
for d in json_data["content"]["boardData"]["indicators"]:
    item = {
        "id": d["id"],
        "title": d["title"],
        **d["indicatorProperties"],
    }
    indicators.append(item)

df_ind = pd.DataFrame(indicators).set_index("id")

# Harmonisera enhet
# En del indikatorer är ton 
df_ind["co2ePotential"] = df_ind["co2ePotential"] * df_ind["co2eUnit"].apply(lambda x: {"Mton": 1, "ton": 1 / 10e6}.get(x))


def interpolate_potential_curve(list_of_dicts):
    df_ts = pd.DataFrame(list_of_dicts).set_index("date")
    df_ts.index = pd.to_datetime(df_ts.index)
    if "value" not in df_ts.columns:
        df_ts["value"] = None
        df_ts["co2e"] = None
    else:
        df_ts["value"] = df_ts["value"].interpolate()
        df_ts["co2e"] = df_ts["co2e"].interpolate()

    return df_ts

# Data för varje år
df_ind["potentialCurve"] = df_ind["potentialCurve"].apply(interpolate_potential_curve)



def get_outcome_data(l):
    s = pd.DataFrame(l).set_index("date")["values"]
    s.index = pd.to_datetime(s.index)
    # [ 2.2 ] = >2.2
    s = s.str[0].dropna()

    if len(s) > 0:
        resp = {
            "LatestValue": s.iloc[-1],
            "LatestTimepoint": s.index[-1],
        }
        if len(s) > 1:
            resp["PrevValue"] = s.iloc[-2]
            resp["PrevTimepoint"] = s.index[-2]
    else:
        resp = {}


    return pd.Series(resp)

# Hämta senast observerade data (samt föregående)
_df_latest_outcome = df_ind["outcome"].apply(get_outcome_data).add_prefix("outcome")
df_ind = df_ind.join(_df_latest_outcome)

def get_latest_potential(row):
    latest_tp = row["outcomeLatestTimepoint"]
    if pd.isnull(latest_tp):
        return None
    else:
        return pd.Series({
            "potentialLatestValue": row["potentialCurve"].loc[latest_tp, "value"],
            "potentialLatestCO2e": row["potentialCurve"].loc[latest_tp, "co2e"],
        })

# Hämta det värde i "potentialValue" som korresponderar till senaste observerade data
_df_latest_potential =  df_ind.apply(get_latest_potential, axis=1)

df_ind = df_ind.join(_df_latest_potential)

df_ind["outcomeVsPotential"] = df_ind["outcomeLatestValue"] - df_ind["potentialLatestValue"]

df_ind.iloc[1].head(30)



KeyError: 'indicators'

title                   Andel elanvändning
outcomeLatestValue                   26.76
potentialLatestValue                 30.27
outcomeVsPotential                   -3.51
goalValue                              NaN
diff                                 -3928
co2ePotential                        0.117
Name: 37, dtype: object

In [45]:
base_url = "https://app.climateview.global/public/board/ec2d0cdf-e70e-43fb-85cb-ed6b31ee1e09?id="
df["url"] = base_url + df["id"]


In [117]:
# parse indicator data
nodes = []
for d in json_data["content"]["boardData"]["nodes"]:
    item = {
        "id": d["id"],
        "title": d["title"],
        **d["nodeProperties"],
    }
    if len(d["indicators"]) > 0:
        d["indicator"] = d["indicators"][0]
        assert len(d["indicators"]) == 1

    nodes.append(item)

df_nodes = pd.DataFrame(nodes).set_index("id")


df_nodes.loc["206f6b8d-c66c-477f-bed1-0204ccfad522"].customTargetModelData

def parse_model_data(json_blob):
    if pd.isna(json_blob):
        return pd.Series({})
    s_outcome = pd.Series(json_blob["chartData"]["outcome"])
    s_outcome.index = s_outcome.index.astype(int)

    if len(s_outcome) == 0:
        return pd.Series({})
    
    

    latest_tp = s_outcome.index.max()

    # interpolera target-serien för att få data för alla år
    s_target = pd.Series(json_blob["chartData"]["target"])
    s_target.index = s_target.index.astype(int)
    years = list(range(s_target.index[0], s_target.index[-1] + 1))
    s_target = s_target.reindex(years).interpolate()

    
    resp = {
        "latestTimepoint": latest_tp,
        "latestOutcome": s_outcome[latest_tp],
        "latestTarget": s_target.get(latest_tp),
    }
    if resp["latestTarget"]:
        resp["diff"] = s_outcome[latest_tp] - s_target[latest_tp]
        resp["diffPct"] = s_outcome[latest_tp] / s_target[latest_tp] - 1
    return pd.Series(resp)

df_outcome_vs_target = df_nodes.customTargetModelData.apply(parse_model_data)

#_df[_df.latestTimepoint.notna()].sort_values("diffPct")

df_nodes = pd.concat([
    df_nodes.drop("diff", axis=1),
    df_outcome_vs_target,
], axis=1)
df_nodes["outcomeCategory"] = df_nodes["diffPct"].apply(lambda x: "good" if x > 0 else "bad")
df_nodes["diffPctWeighted"] = df_nodes["co2e"] * df_nodes["diffPct"]

has_data = df_nodes.diffPct.notna()
main_cols = ["title", "co2e", "latestOutcome", "latestTarget", "diffPct"]
df_nodes[has_data][main_cols].to_csv("~/Downloads/panorma-nodes-outcome-vs-target-3.csv")
df_nodes[has_data].sort_values("diffPctWeighted")

/tmp/ipykernel_45146/849512500.py:22: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.Series({})
/tmp/ipykernel_45146/849512500.py:23: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s_outcome = pd.Series(json_blob["chartData"]["outcome"])
/tmp/ipykernel_45146/849512500.py:27: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.Series({})


,title,icon,type,imgURL,thumbnailURL,description,co2e,data,diagramHeader,extendedDescription,...,transitionTargetData,firebaseId,achivableLater,latestTimepoint,latestOutcome,latestTarget,diff,diffPct,outcomeCategory,diffPctWeighted
id,,,,,,,,,,,,,,,,,,,,,
7628b744-d658-4ca8-91ce-6f105d81dd45,Biobränsle istället för kol,/assets/images/transition-elements-icons/biofu...,5,,,,0.578,[],Möjlig utsläppsminskning,NaN,...,NaN,NaN,NaN,2020.0,0.00000,0.785213,-0.785213,-1.000000,bad,-0.578000
1c910ad7-5124-4d9f-9758-a9a0a19e7f37,Biobränsle för cement och mineralindustri,/assets/images/transition-elements-icons/biofu...,5,,,,0.408,[],Möjlig utsläppsminskning,NaN,...,NaN,NaN,NaN,2019.0,5.78216,27.488208,-21.706047,-0.789649,bad,-0.322177
b61ae7e0-6b0b-4ecd-88eb-e853f292f714,Minskad användning av lösningsmedel,/assets/images/transition-elements-icons/reduc...,5,,,,0.326,[],Möjlig utsläppsminskning,NaN,...,NaN,NaN,NaN,2019.0,3.00000,36.514286,-33.514286,-0.917840,bad,-0.299216
8b301e8b-5b22-4124-97d6-e7d95b6f741e,Förnybara råvaror,/assets/images/transition-elements-icons/renew...,5,,,,1.775,[],Möjlig utsläppsminskning,NaN,...,NaN,NaN,NaN,2021.0,2.50000,3.000000,-0.500000,-0.166667,bad,-0.295833
ab98a4aa-a42c-4375-a758-625e714c84aa,Elektrifiering av produktion,/assets/images/transition-elements-icons/elect...,5,,,,0.289,[],Möjlig utsläppsminskning,NaN,...,NaN,NaN,NaN,2018.0,0.00000,7.620000,-7.620000,-1.000000,bad,-0.289000
ebb72100-0b8d-4117-b652-a7b2593c179d,Biodrivmedel inrikes sjöfart,/assets/images/transition-elements-icons/biofu...,5,,,,0.358,[],Möjlig utsläppsminskning,NaN,...,"{'transitionProperties': {'targetLevel': 0.25,...",NaN,NaN,2020.0,5.10000,24.750000,-19.650000,-0.793939,bad,-0.284230
0328c7f3-a938-42c4-a62c-b238180ddd27,Fossilfri vätgasproduktion,/assets/images/transition-elements-icons/renew...,5,,,,0.282,[],Möjlig utsläppsminskning,NaN,...,NaN,NaN,NaN,2021.0,0.00000,1082.000000,-1082.000000,-1.000000,bad,-0.282000
8f3e69ba-6ffd-46ca-a493-b7b627fe0481,Från betong/stål till trästomme i flerbostadshus,/assets/images/transition-elements-icons/low_c...,5,,,,0.326,[],Möjlig utsläppsminskning,NaN,...,"{'transitionProperties': {'targetLevel': 0.25,...",NaN,NaN,2020.0,4110.00000,28052.000000,-23942.000000,-0.853486,bad,-0.278237
cc3d3604-ff91-400b-937d-20aec0fd28bc,Ökad andel kollektivtrafik,/assets/images/transition-elements-icons/defau...,5,,,,0.809,[],Möjlig utsläppsminskning,{'data': {'texts': [{'title': 'Mer kollektivtr...,...,NaN,NaN,NaN,2021.0,8.47000,12.526000,-4.056000,-0.323806,bad,-0.261959


In [136]:
indicator_cols = ["title", "outcomeLatestValue", "potentialLatestValue", "outcomeVsPotential", "goalValue", "diff", "co2ePotential"]
df_ind[indicator_cols].columns.map(lambda x: "Indicator" + x.title())

["title", ]
df_nodes.columns

Index(['title', 'icon', 'type', 'imgURL', 'thumbnailURL', 'description',
       'co2e', 'data', 'diagramHeader', 'extendedDescription',
       'achievableLater', 'notAchievable', 'groupColour',
       'historicalEmissionData', 'modelType', 'customTargetModelData',
       'transitionTargetDescription', 'transitionTargetSummary', 'status',
       'transitionTarget', 'transitionElement', 'transitionTargetData',
       'firebaseId', 'achivableLater', 'latestTimepoint', 'latestOutcome',
       'latestTarget', 'diff', 'diffPct', 'outcomeCategory',
       'diffPctWeighted'],
      dtype='object')

In [114]:
df_nodes.loc["40740dea-2ece-447a-995a-67ff093828c7"]

title                                                 Överflyttning till järnväg
icon                           /assets/images/transition-elements-icons/trans...
type                                                                           5
imgURL                                                                          
thumbnailURL                                                                    
description                                                                     
co2e                                                                       0.607
data                                                                          []
diagramHeader                                           Möjlig utsläppsminskning
extendedDescription            {'data': {'texts': [{'title': 'Åka tåg'}], 'im...
achievableLater                                                            False
notAchievable                                                              False
groupColour                 